In [2]:
import numpy as np
import tensorflow as tf
import pydicom
import os
from skimage import measure, morphology
from scipy.ndimage import zoom
from datetime import datetime

# Función para crear un volumen 3D desde múltiples archivos DICOM
def create_3d_volume(dicom_files):
    slices = []
    spacings = []
    for dicom_path in dicom_files:
        try:
            dicom = pydicom.dcmread(dicom_path)
            image = dicom.pixel_array.astype(np.int16)

            # Normalizar valores HU
            image[image == -2000] = 0
            intercept = dicom.RescaleIntercept
            slope = dicom.RescaleSlope
            image = image * slope + intercept
            slices.append(image)

            # Extraer la resolución del voxel
            pixel_spacing = getattr(dicom, "PixelSpacing", [1, 1])
            slice_thickness = getattr(dicom, "SliceThickness", 1)
            spacings.append(list(pixel_spacing) + [slice_thickness])

        except Exception as e:
            print(f"Error al procesar el archivo {dicom_path}: {e}")
            continue

    slices = np.array(slices)
    spacings = np.mean(spacings, axis=0) if spacings else [1, 1, 1]
    return np.stack(slices, axis=0), spacings

# Función para segmentar los pulmones
def segment_lungs(volume):
    binary_mask = (volume > -700) & (volume < -600)
    binary_mask = morphology.remove_small_objects(binary_mask, min_size=500)

    # Etiquetar regiones conectadas
    labeled_mask = measure.label(binary_mask, connectivity=1)
    mask = morphology.remove_small_objects(labeled_mask, min_size=1000)

    # Aplicar la máscara al volumen original
    segmented_volume = volume * (mask > 0)
    return segmented_volume, mask

# Función para generar un mensaje HL7
def generate_hl7_message(patient_id, diagnosis, probability_fibrosis, probability_no_fibrosis):
    hl7_message = f"""MSH|^~\&|RadiologyDepartment|HospitalSystem|HealthSystem|HealthcareProvider|{datetime.now().strftime('%Y%m%d%H%M%S')}||ORU^R01|{patient_id}|P|2.3
PID|1||{patient_id}||Ejemplo de Paciente||19850101|M||| 
OBR|1|{patient_id}|12345|CT^Chest|Chest CT|{datetime.now().strftime('%Y%m%d%H%M%S')}||| 
OBX|1|TX|Diagnosis^Fibrosis_Pulmonar_Idiopática|1|{diagnosis}|{probability_fibrosis:.2f}%|{probability_no_fibrosis:.2f}%|F|||F
OBX|2|TX|Diagnosis^No_Fibrosis|2|No Fibrosis Pulmonar Idiopática|{probability_no_fibrosis:.2f}%|{probability_fibrosis:.2f}%|N|||F
"""
    return hl7_message


# Función para predecir el diagnóstico de un paciente sin bloques
def predict_patient_diagnosis(patient_dir, patient_id, model_path="phase_2_model.h5"):
    dicom_files = [os.path.join(patient_dir, f) for f in os.listdir(patient_dir) if f.endswith('.dcm')]
    
    # Crear volumen 3D
    volume, spacing = create_3d_volume(dicom_files)

    # Segmentar pulmones
    segmented_volume, mask = segment_lungs(volume)

    # Asegurar que el volumen tenga la forma correcta (64, 64, 64)
    target_shape = (64, 64, 64)
    
    # Redimensionar el volumen para que tenga el tamaño adecuado (si es necesario)
    if segmented_volume.shape != target_shape:
        zoom_factors = [t / s for t, s in zip(target_shape, segmented_volume.shape)]
        segmented_volume = zoom(segmented_volume, zoom_factors, order=1)

    # Asegurarse de que el volumen tenga la forma (64, 64, 64, 1) para el modelo
    volume_input = np.expand_dims(segmented_volume, axis=-1)  # Añadir la dimensión del canal
    volume_input = np.expand_dims(volume_input, axis=0)  # Añadir la dimensión del batch (1 volumen)

    # Cargar el modelo
    model = tf.keras.models.load_model(model_path)

    # Realizar la predicción
    prediction = model.predict(volume_input)

    # Interpretar la predicción
    diagnosis = "No Fibrosis Pulmonar Idiopática" if np.argmax(prediction) == 1 else "Fibrosis Pulmonar Idiopática"
    probability_fibrosis = prediction[0][0] * 100
    probability_no_fibrosis = prediction[0][1] * 100

    # Generar el mensaje HL7
    hl7_message = generate_hl7_message(patient_id, diagnosis, probability_fibrosis, probability_no_fibrosis)
    
    # Imprimir el diagnóstico en formato legible
    print(f"Diagnóstico: {diagnosis}")
    print(f"Probabilidad de Fibrosis Pulmonar Idiopática: {probability_fibrosis:.2f}%")
    print(f"Probabilidad de No Fibrosis Pulmonar Idiopática: {probability_no_fibrosis:.2f}%\n")

    # Imprimir el mensaje HL7
    print("Mensaje HL7:")
    print(hl7_message)

# Ruta al directorio del paciente (un solo paciente)
patient_dir = "C:/Users/Marcos/reto-1-mas-alla-de-la-mirada-humana/ID00407637202308788732304"
patient_id = "ID00407637202308788732304"  # Usamos el ID del paciente

# Realizar la predicción y generar el mensaje HL7
predict_patient_diagnosis(patient_dir, patient_id)

<>:58: SyntaxWarning: invalid escape sequence '\&'
<>:58: SyntaxWarning: invalid escape sequence '\&'
C:\Users\Marcos\AppData\Local\Temp\ipykernel_12668\2434396150.py:58: SyntaxWarning: invalid escape sequence '\&'
  """


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Diagnóstico: Fibrosis Pulmonar Idiopática
Probabilidad de Fibrosis Pulmonar Idiopática: 52.12%
Probabilidad de No Fibrosis Pulmonar Idiopática: 47.88%

Mensaje HL7:
MSH|^~\&|RadiologyDepartment|HospitalSystem|HealthSystem|HealthcareProvider|20241124134311||ORU^R01|ID00407637202308788732304|P|2.3
PID|1||ID00407637202308788732304||Ejemplo de Paciente||19850101|M||| 
OBR|1|ID00407637202308788732304|12345|CT^Chest|Chest CT|20241124134311||| 
OBX|1|TX|Diagnosis^Fibrosis_Pulmonar_Idiopática|1|Fibrosis Pulmonar Idiopática|52.12%|47.88%|F|||F
OBX|2|TX|Diagnosis^No_Fibrosis|2|No Fibrosis Pulmonar Idiopática|47.88%|52.12%|N|||F

